In [ ]:

# This program is meant to help save the bursts in a large dataframe/excel file containing a lot of noise
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from os import listdir
import os
from os.path import isfile, join
import shutil
from scipy import signal
import sys
# from tsfresh import extract_features
# from tsfresh import select_features
# from tsfresh.utilities.dataframe_functions import impute
# from tsfresh import extract_relevant_features


noise = -85

class Peak:
    def __init__(self, start, end, height):
        self.start = start
        self.end = end
        self.width = end - start
        self.height = height
    def getStart(self):
        return self.start
    def getEnd(self):
        return self.end
    def getWidth(self):
        return self.width
    def getHeight(self):
        return self.height               
    def __str__(self):
        return "START: " + str(self.start) + "; END: " + str(self.end) + "; WIDTH: " + str(self.width) + "; HEIGHT: " + str(self.height)
    def __repr__(self):
        return str(self) + '\n'

    
def amplitude(val):
    global noise
    if val == '0j' or val == '' or val == '0':
        return noise
    comp = complex(val)
    real = comp.real  #extract real part and convert to a float
    imaginary = comp.imag #remove the space between the sign and number for imaginary and convert to float
    
    refLevDBM = 30
    absComplex = abs(complex(real, imaginary))
    
    try:
        sigDB = 20* math.log(absComplex, 10)
        
    except:
        print("val is " + str(val))
        print("abscomplex is " + str(absComplex))
        print(sigDB)
        sys.exit()
        
    sigDBM = sigDB + 30 - refLevDBM
    
    return sigDBM #complex takes two floats, real and imaginary, and creates a complex number, return that complex number

# what I want to do is write something which tells me when the slope has suddenly changed....
# alternatively I could have something which tells me when the mean amplitude has changed,
# tells me the exact location for the change if the average is above noise for 4000 samples
# add start end and width to possible peaks
def identifyPeaks(data):
    df = data.copy()
    
    numSamples = df.index.max()
    jumpBy = 1000
    roll = 100
    df['Amplitude'] = df['Amplitude'].rolling(roll).mean(center = True)
    i = 0
    peaks = []
    
    if numSamples < minWidth:
        return []
    
    while i < df.index.max():
        end = 0
        start = 0
        amp = df['Amplitude'][i]
        if amp >= noise:
            while amp >= noise and i >= 0: #looks for peak's exact start
                i-=1
                amp = df['Amplitude'][i]
                
            start = i
            amp = df['Amplitude'][i+1]
            while amp >= noise and i + 100 <= numSamples :
                i+=100
                amp = df['Amplitude'][i]
                
            while amp <= noise and i >= 0:
                i-=1
                amp = df['Amplitude'][i]
                end = i
                
            if end - start >= minWidth and end != 0 and start != roll - 2:
                
                plt.plot(data['Amplitude'][max(0, start - 200) : min(end + 200, numSamples)])
                plt.savefig(savePlotsTo + '//' + device + '//' +str(burstNum)+ "_" + str(len(peaks)))
                plt.cla()
                
                center = (start + end) // 2
                centerLeft = (center + start)//2
                centerRight = (center + end)//2
            
                height = df['Amplitude'][centerLeft: centerRight].mean()
                newPeak = Peak(start, end, height)
                peaks.append(newPeak)
            else:
                print(end)
                print(start)
        i+=jumpBy
    
#     print(peaks)
    
    if len(peaks) > 2:
        peaks = twoHighest(peaks)
    
    return peaks

def twoHighest(peaks):
    
    if len(peaks) == 2:
        return peaks
    
    elif len(peaks) == 1:
        zeroPeak = Peak(0, 0, 0)
        peaks.append(zeroPeak)
        return peaks
    
    elif len(peaks) == 0:
        zeroPeak = Peak(0, 0, 0)
        zeroPeak2 = Peak(0, 0, 0)
        peaks.append(zeroPeak)
        peaks.append(zeroPeak2)
        return peaks
    
    else:
        peaks.sort(key = Peak.getHeight)
#         print(peaks)
        peaks = peaks[-2:]
        peaks.sort(key = Peak.getStart)
        return peaks

def getDeviceName(subdir):
    i = -1
    device = ''
    while abs(i) < len(subdir):
        if not subdir[i] == '/':
            device += subdir[i]
            i -= 1
        else:
            return device[::-1]
        
def getBurstNum(file):
    i = -1
    num = ''
    
    while abs(i) < len(file):
        if not file[i] == '.':
            i -= 1
            
        else:
            i-=1
            while abs(i) < len(file) and file[i] != 't': 
                num += file[i]
                i-=1
                    
            return int(num[::-1])
        
        
def deviceArr(path):
    
    devices = []
    
    subdirs = [f.path for f in os.scandir(path) if f.is_dir() ]
    
    i = 0
    for subdir in subdirs:
        devices.insert(i, getDeviceName(subdir))
        i+=1
    return devices

def makeDir(pathWay):
    if os.path.isdir(pathWay):
        shutil.rmtree(pathWay)
        
    os.mkdir(pathWay)

if __name__ == "__main__":
    # assumptions, there is a directory which has folders labelled by device name. The folder contains amplitude information. 
    # requirements: must be able to pick up the two strongest plateaus
    # will use python peak detection to find out the two or one peaks worth looking at. Will look at the most prominent peaks
    # first.
    path = r'AsExcel'
    savePlotsTo = r'PlotsAll'
    newBursts = r'smallBursts'
    samplingRate = 112*(10**6) # sampling rate of collection device in samples/second
    minLength = 40/(10**6) # min length of a bluetooth packet (8us for preamble and 32us for access address) in seconds
    minWidth = minLength * samplingRate #minwidth is 4480samples
    
    devices = deviceArr(path)
    
    subdirs = [f.path for f in os.scandir(path) if f.is_dir() ]
    
    makeDir(savePlotsTo)
    makeDir(newBursts)
    
    
    for device in devices:
        makeDir(savePlotsTo + '/' + device) #make folder to save plots to
        makeDir(newBursts + '/' + device)
        
        print(device)
        subdir = path + '/' + device
        
        bursts = [f for f in listdir(subdir) if isfile(join(subdir, f))]
        
        size = len(bursts) * 2 # we expect each burst to contain 2 bursts
        
        df = pd.DataFrame(index = range(size), columns = ['Time', 'Amplitude', 'PeakNumber'])
        i = 0
        for file in bursts:
            burstNum = getBurstNum(file)
            fileDir = path + '/' + device + '/' + file
            print(fileDir)
            data = pd.read_excel(fileDir)
            data = data.rename(columns = {0 : 'Amplitude'})
            data['Amplitude'] = data['Amplitude'].apply(amplitude) # convert to amplitude
            data = data.rename(columns = {'Unnamed: 0': 'Time'})
            data['Time'] = data.index
            data['Device'] = device
            
            peaks = identifyPeaks(data)
            toSave = pd.DataFrame(columns = df.columns)
            
            
            label = '(a)'
            for peak in peaks:
                
                
                toSave = data[peak.start : peak.end]
                
                toSave.to_excel(newBursts + '/' + device + '/' + file[:-5] + label + '.xlsx')
                label = '(b)'
                
            plt.plot(data['Amplitude'])
            plt.savefig(savePlotsTo + '/' + device + '/' + str(burstNum))
            plt.cla()
            
            



iHealth Blood Pressure1
AsExcel/iHealth Blood Pressure1/Burst114.xlsx
AsExcel/iHealth Blood Pressure1/Burst051.xlsx
AsExcel/iHealth Blood Pressure1/Burst143.xlsx
AsExcel/iHealth Blood Pressure1/Burst006.xlsx
AsExcel/iHealth Blood Pressure1/Burst155.xlsx
AsExcel/iHealth Blood Pressure1/Burst010.xlsx
AsExcel/iHealth Blood Pressure1/Burst102.xlsx
AsExcel/iHealth Blood Pressure1/Burst047.xlsx
AsExcel/iHealth Blood Pressure1/Burst250.xlsx
AsExcel/iHealth Blood Pressure1/Burst196.xlsx
AsExcel/iHealth Blood Pressure1/Burst179.xlsx
AsExcel/iHealth Blood Pressure1/Burst207.xlsx
AsExcel/iHealth Blood Pressure1/Burst084.xlsx
AsExcel/iHealth Blood Pressure1/Burst211.xlsx
141211
141039
143444
143098
145539
145341
AsExcel/iHealth Blood Pressure1/Burst138.xlsx
AsExcel/iHealth Blood Pressure1/Burst092.xlsx
AsExcel/iHealth Blood Pressure1/Burst246.xlsx
AsExcel/iHealth Blood Pressure1/Burst180.xlsx
AsExcel/iHealth Blood Pressure1/Burst118.xlsx
AsExcel/iHealth Blood Pressure1/Burst231.xlsx
AsExcel/iHealt